## Criteria evaluator

### Criteria evaluator

In [ ]:
from langchain.evaluation import load_evaluator

# Wczytujemy evaluator kryterialny
evaluator = load_evaluator("criteria", criteria="correctness")

# Porównujemy odpowiedź z pytaniem
result = evaluator.evaluate_strings(
    prediction="2 + 2 = 5",
    input="Policz 2 + 2",
    reference="4"
)

print(result)  # {'score': 1, 'reasoning': 'Odpowiedź jest błędna'}


### Custom evaluator

In [ ]:
import os
from langsmith import Client
from langchain_openai import ChatOpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "GEMMA EVALUATIONs"
client = Client()
# Inputs are provided to your model, so it know what to generate
dataset_inputs = [
    "Why people don't have 3 legs?",
    "Why people are not flying?",
]

#use 1st LLM for generating texts
llm_test= ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1,max_tokens=256)
# llm_gen = ChatOpenAI(base_url="https://api-inference.huggingface.com/v1", model="google/gemma-2b-it",temperature=0.1,max_tokens=256)
# and 2nd to evaluate different criteria of response generated with 1st LLM
llm_gen = ChatOpenAI(model="gpt-4o", temperature=0.1,max_tokens=256)

dataset_outputs = [
    {"result": llm_test.invoke(50*dataset_inputs[0])},
    {"result": llm_test.invoke(50*dataset_inputs[1])},
]
print(dataset_outputs)

In [1]:
from langchain.smith import RunEvalConfig
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def custom_evaluator(run, example) -> EvaluationResult:
    """
    checks if output contains specific word
    :param run:
    :param example:
    :return: int
    """
    generated = run.outputs["generations"][0][0]["text"]
    if 'human' in generated:
        score = 1
    else:
        score = 0
    return EvaluationResult(key="result", score=score)

In [2]:
eval_config = RunEvalConfig(
    custom_evaluators=[custom_evaluator],
    evaluators=[
        "criteria",
        "qa",         #directly grade a response as "correct" or "incorrect" based on the reference answer
        "context_qa", #use the provided reference context in determining correctness
        "cot_qa",     #use chain of thought "reasoning" before determining a final verdict. This tends to lead to responses that better correlate with human labels
        RunEvalConfig.Criteria("insensitivity"),
        RunEvalConfig.Criteria("relevance"),
        RunEvalConfig.Criteria("helpfulness"),
        RunEvalConfig.Criteria("maliciousness"),
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria("coherence"),
        RunEvalConfig.Criteria("conciseness"),
        RunEvalConfig.Criteria("misogyny"),
        RunEvalConfig.Criteria("criminality"),
        RunEvalConfig.Criteria("controversiality"),
        RunEvalConfig.Criteria( #custom defined criteria related to specific problem we want to solve and problems detected in output
            {
                "valuation": "Do texts contain valuation of subject, like glorifying some characteristic or judging someone?"
                " Respond Y if they do, N if they're entirely objective and stick to the facts without additions."
            }
        )
    ],
)

In [ ]:
import uuid

dataset_name = "existential questions run:" + uuid.uuid4().__str__() #need to modify this value on every run of notebook

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="evaluate LLM output",
)
client.create_examples(
    inputs=[{"question": q} for q in dataset_inputs],
    outputs=dataset_outputs,
    dataset_id=dataset.id,
)

In [ ]:
#in case of error ‘model is currently loading;’, wait couple of minutes and run notebook again
scores = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=llm_gen,
    evaluation=eval_config,
    verbose=True,
    project_name=dataset_name,
)
print(scores)

### Ewaluacja LangSmith
Poniższy przykład pochodzi z LangSmith

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
# Włącz śledzenie LangSmith (wymaga konta LangSmith):
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "kurs-demo"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGSMITH_API_KEY"] = "<TWÓJ_KLUCZ>" #załączony w .env

In [9]:
from langsmith import Client, wrappers
from openevals.llm import create_llm_as_judge
from openevals.prompts import CORRECTNESS_PROMPT
from openai import OpenAI
from random import randint

# Define the input and reference output pairs that you'll use to evaluate your app
client = Client()

# Create the dataset
dataset = client.create_dataset(
    dataset_name=f"Sample dataset {randint(1, 100000)}", description="A sample dataset in LangSmith."
)

# Create examples in the dataset. Examples consist of inputs and reference outputs
examples = [
    {
        "inputs": {"question": "Which country is Mount Kilimanjaro located in?"},
        "outputs": {"answer": "Mount Kilimanjaro is located in Tanzania."},
    },
    {
        "inputs": {"question": "What is Earth's lowest point?"},
        "outputs": {"answer": "Earth's lowest point is The Dead Sea."},
    },
]

# Add the examples to the dataset
client.create_examples(dataset_id=dataset.id, examples=examples)


{'example_ids': ['8176bb46-94df-4304-aee9-75126316b554',
  '23eca3dd-54c1-4757-ba28-b1f2cc85b8de'],
 'count': 2}

In [10]:
# Wrap the OpenAI client for LangSmith tracing
openai_client = wrappers.wrap_openai(OpenAI())

# Define the application logic you want to evaluate inside a target function. For example, this may be one LLM call that includes the new prompt you are testing, a part of your application or your end to end application
# The SDK will automatically send the inputs from the dataset to your target function
def target(inputs: dict) -> dict:
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Answer the following question accurately"},
            {"role": "user", "content": inputs["question"]},
        ],
    )
    return { "answer": response.choices[0].message.content.strip() }

In [11]:
# Define an LLM as a judge evaluator to evaluate correctness of the output
# Import a prebuilt evaluator prompt from openevals (https://github.com/langchain-ai/openevals) and create an evaluator.

def correctness_evaluator(inputs: dict, outputs: dict, reference_outputs: dict):
    evaluator = create_llm_as_judge(
        prompt=CORRECTNESS_PROMPT,
        model="openai:o3-mini",
        feedback_key="correctness",
    )
    eval_result = evaluator(
        inputs=inputs,
        outputs=outputs,
        reference_outputs=reference_outputs
    )
    return eval_result

In [7]:
# After running the evaluation, a link will be provided to view the results in langsmith
experiment_results = client.evaluate(
    target,
    data="Sample dataset",
    evaluators=[
        correctness_evaluator,
        # you can add multiple evaluators here
    ],
    experiment_prefix="first-eval-in-langsmith",
    max_concurrency=2,
)

View the evaluation results for experiment: 'first-eval-in-langsmith-894355be' at:
https://smith.langchain.com/o/3e1f981e-76ef-5491-9a42-e33f3bdfeba4/datasets/ea300447-dba3-4535-8eb3-3127e167808d/compare?selectedSessions=cc4fd684-a692-4450-82c3-4d9c6bde46ea




0it [00:00, ?it/s]